## IMPORTS

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
drive.mount('/content/gdrive')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Import necessary libraries
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
trainFolder = "gdrive/My Drive/MLDM - Carrefour Project/data-train"

import os
for dirname, _, filenames in os.walk(f'{trainFolder}'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Mounted at /content/gdrive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

gdrive/My Drive/MLDM - Carrefour Project/data-train/products_data.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/test_data.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_1.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_10.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_2.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_3.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_4.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_5.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_6.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_7.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_8.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_9.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/products_embe.csv


In [ ]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### Data Description

This project uses three datasets:

#### `train_data.csv`:
This dataset contains two years (2022 & 2023) of historical transactions for 100,000 Carrefour customers. It has 10 columns:

* ***date***: Date of the transaction
* ***transaction_id***: ID of the transaction
* ***customer_id***: Customer ID
* ***product_id***: Product purchased
* ***has_loyality_card***: Flag indicating whether the customer has a loyalty card
* ***store_id***: Store where the purchase was made
* ***is_promo***: Flag indicating whether there was a discount on the product
* ***quantity***: Quantity purchased of the product
* ***format***: Ecommerce activity format (clcv, lex, or DRIVE)
  - clcv : courses livrées chez vous
  - lex : livraison express
  - DRIVE.
* ***orderChannelCode***: Indicates whether the online activity was made through the website or mobile app

#### `products_data.csv`:
This dataset contains detailed information about the products. The following columns are relevant to this project:

* ***product_id*** : Product name
* ***product_description*** : Product description
* ***department_key***: Department key
* ***class_key***: Class key
* ***subclass_key***: Subclass key
* ***sector***: sector name
* ***brand_key***: Brand name
* ***shelf_level1***: Top-level shelf category
* ***shelf_level2***: Second-level shelf category
* ***shelf_level3***: Third-level shelf category
* ***shelf_level4***: Fourth-level shelf category
* ***sector***: Sector
* ***bio***: Flag indicating whether the product is organic
* ***sugar_free***: Flag indicating whether the product is sugar-free
* ***aspartame_free***: Flag indicating whether the product is aspartame-free
* ***gluten_free***: Flag indicating whether the product is gluten-free
* ***halal***: Flag indicating whether the product is halal
* ***casher***: Flag indicating whether the product is kosher
* ***eco_friendly***: Flag indicating whether the product is eco-friendly
* ***local_french***: Flag indicating whether the product is locally produced in France
* ***artificial_coloring_free***: Flag indicating whether the product is free of artificial coloring
* ***taste_enhancer_free***: Flag indicating whether the product is free of taste enhancers
* ***naturality***: Naturality score
* ***antibiotic_free***: Flag indicating whether the product is antibiotic-free
* ***reduced_sugar***: Flag indicating whether the product has reduced sugar content
* ***vegetarian***: Flag indicating whether the product is vegetarian
* ***pesticide_free***: Flag indicating whether the product is pesticide-free
* ***grain_free***: Flag indicating whether the product is grain-free
* ***no_added_sugar***: Flag indicating whether the product has no added sugar
* ***salt_reduced***: Flag indicating whether the product has reduced salt content
* ***nitrite_free***: Flag indicating whether the product is nitrite-free
* ***fed_without_ogm***: Flag indicating whether the animals were fed without GMOs
* ***no_added_salt***: Flag indicating whether the product has no added salt
* ***no_artificial_flavours***: Flag indicating whether the product has no artificial flavors
* ***porc***: Flag indicating whether the product contains pork
* ***vegan***: Flag indicating whether the product is vegan
* ***frozen***: Flag indicating whether the product is frozen
* ***fat_free***: Flag indicating whether the product is fat-free
* ***reduced_fats***: Flag indicating whether the product has reduced fat content
* ***fresh***: Flag indicating whether the product is fresh
* ***alcool***: Flag indicating whether the product contains alcohol
* ***lactose_free***: Flag indicating whether the product is lactose-free
* ***phenylalanine_free***: Flag indicating whether the product is phenylalanine-free
* ***palm_oil_free***: Flag indicating whether the product is palm oil-free
* ***ecoscore***: Ecoscore
* ***produits_du_monde***: Flag indicating whether the product is an international product
* ***regional_product***: Flag indicating whether the product is a regional product
* ***national_brand***: Flag indicating whether the product is a national brand
* ***first_price_brand***: Flag indicating whether the product is a first-price brand
* ***carrefour_brand***: Flag indicating whether the product is a Carrefour brand

#### `test_data.csv`:
This dataset contains the actual purchases of the first 80,000 customers in 2024. It has three columns:

* ***transaction_id***: ID of the transaction
* ***customer_id***: Customer ID
* ***product_id***: the id of the purchased product

### Load data

* Load *train_data.csv*, *products_data.csv* and *test_data.csv* using pandas.

In [ ]:
# This code reads the data from CSV files named "train_data_part_i.csv" for all i from 1 to 10
# and concatenates them into a single pandas DataFrame
train_dataframes = []
for i in tqdm(range(1, 11)): ## 1 - 11
    train_dataframes.append(pd.read_csv(f'{trainFolder}/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

100%|██████████| 10/10 [01:59<00:00, 11.92s/it]


In [ ]:
len(train_data)

87037462

In [ ]:
# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv(f'{trainFolder}/products_data.csv')

<ipython-input-5-d8cc84ca7717>:2: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv(f'{trainFolder}/products_data.csv')


In [ ]:
# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv(f'{trainFolder}/test_data.csv')

In [ ]:
products_data_copy = pd.read_csv(f'{trainFolder}/products_embe.csv')

In [ ]:
products_data_copy = products_data_copy[['product_id', 'embedding']]
products_data_copy

,product_id,embedding
0,Product_33508,[-4.32211161e-01 -2.05061417e-02 2.77520180e-...
1,Product_15347,[-3.59862775e-01 -1.68674756e-02 2.61252761e-...
2,Product_80604,[-3.07046682e-01 1.69049855e-02 3.06277961e-...
3,Product_81740,[-4.70893383e-01 7.09071010e-02 1.97719961e-...
4,Product_82516,[-4.24364060e-01 -2.90127173e-02 2.67759621e-...
...,...,...
82961,Product_18949,[-1.78783506e-01 1.82161704e-01 2.99285829e-...
82962,Product_66524,[-1.18125834e-01 1.75526097e-01 3.60967815e-...
82963,Product_66126,[-3.55841406e-02 1.70518711e-01 1.66674897e-...
82964,Product_31161,[-2.96732664e-01 9.20548663e-02 1.73864007e-...


In [ ]:
train_data.sample(5)

,date,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel
38506477,2022-10-15,Transaction_2017823,Household_42211,Product_12269,0,Store_1445,0,1.0,DRIVE,MOBILE_APP
78443391,2023-03-09,Transaction_2545323,Household_84040,Product_70416,0,Store_2372,0,1.0,DRIVE,MOBILE_APP
33379662,2023-10-16,Transaction_580981,Household_38936,Product_16477,0,Store_631,0,1.0,DRIVE,MOBILE_APP
25277279,2022-11-01,Transaction_2007391,Household_23413,Product_41635,0,Store_1670,0,1.0,CLCV,MOBILE_APP
41435813,2023-09-09,Transaction_2516063,Household_45585,Product_31910,0,Store_1670,1,2.0,CLCV,WEBSITE


In [ ]:
products_data.sample(3)

,product_id,product_description,department_key,class_key,subclass_key,sector,brand_key,shelf_level1,shelf_level2,shelf_level3,shelf_level4,bio,sugar_free,aspartame_free,gluten_free,halal,casher,eco_friendly,local_french,artificial_coloring_free,taste_enhancer_free,naturality,antibiotic_free,reduced_sugar,vegetarian,pesticide_free,grain_free,no_added_sugar,salt_reduced,nitrite_free,fed_without_ogm,no_added_salt,no_artificial_flavours,porc,vegan,frozen,fat_free,reduced_fats,fresh,alcool,lactose_free,phenylalanine_free,palm_oil_free,ecoscore,produits_du_monde,regional_product,national_brand,first_price_brand,carrefour_brand
17603,Product_24118,3X10 D HLW 2FRUITY FRAMB PECHE,Department_14,Class_1424,SubClass_14241,PGC,HOLLYWOOD,Epicerie sucrée,Confiseries et Chocolats,Chewings gums,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0
30314,Product_58432,400G PATE DE CAMPAGNE J.BRIENT,Department_20,Class_2010,SubClass_20103,PRODUITS FRAIS TRANSFORMATION,BRIENT,Charcuterie et Traiteur,Charcuterie à la coupe,"Pâtés, Rillettes et Foies gras",NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,D,0,0,1,0,0
57314,Product_41071,SO-MDREX15LP-INT-NOIR,Department_43,Class_4330,SubClass_43301,EPCS,SONY,Image et Son,Casque et Ecouteurs,Ecouteurs,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0


In [ ]:
test_data.sample(3)

,transaction_id,customer_id,product_id
1131170,Transaction_2024_5302,Household_62471,Product_37298
1110184,Transaction_2024_89554,Household_58387,Product_54058
834893,Transaction_2024_56282,Household_4157,Product_34790


## Create Product Embeddings

In [ ]:
def preprocess_product_data(row):
    # Combine key text features and flag indicators for embedding
    text = f"{row['product_id']} {row['product_description']} " \
           f"Sector: {row['sector']} Brand: {row['brand_key']} " \
           f"Department: {row['department_key']} Class: {row['class_key']} " \
           f"Shelf categories: {row['shelf_level1']} {row['shelf_level2']} {row['shelf_level3']} {row['shelf_level4']}"

    # Append flags with descriptive text if they are set to 1
    flags = [
        ("Organic", row['bio']),
        ("Sugar-free", row['sugar_free']),
        ("Aspartame-free", row['aspartame_free']),
        ("Gluten-free", row['gluten_free']),
        ("Halal", row['halal']),
        ("Kosher", row['casher']),
        ("Eco-friendly", row['eco_friendly']),
        ("Locally produced in France", row['local_french']),
        ("Artificial-coloring-free", row['artificial_coloring_free']),
        ("Taste-enhancer-free", row['taste_enhancer_free']),
        ("Antibiotic-free", row['antibiotic_free']),
        ("Vegan", row['vegan']),
        ("Lactose-free", row['lactose_free']),
        ("Alcohol-free", not row['alcool']), # Inverted for better readability
        ("Eco-score", row['ecoscore'])  # Using ecoscore as-is, assuming it's a score
    ]
    for flag_text, flag_value in flags:
        if flag_value == 1:
            text += f" {flag_text}"

    return text

# Apply this preprocessing to all rows
products_data['combined_text'] = products_data.apply(preprocess_product_data, axis=1)
products_data['combined_text'][0]

'Product_33508 PET 1L LORINA TONIC ARTISANAL Sector: PGC Brand: LORINA Department: Department_10 Class: Class_1000 Shelf categories: Boissons Colas, Thés glacés, Sirops et Sodas Limonades, Limes et Tonics nan Alcohol-free'

In [ ]:
import torch
from tqdm import tqdm  # To display a progress bar

# Define the batch processing function
def get_bert_embeddings_batch(texts, batch_size=16):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        # Tokenize the batch
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=128)

        # Check if CUDA is available before moving to GPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        inputs = {key: value.to(device) for key, value in inputs.items()}


        # Inference with no_grad for faster processing
        with torch.no_grad():
            outputs = model(**inputs)

        # Mean pooling to get a single embedding per input
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Move back to CPU for storage
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

# Check if CUDA is available before moving the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device) # Move model to the appropriate device

# Apply the batch embedding function to products
products_data_copy = products_data.copy()
#products_data_copy = products_data_copy[-1000:]  # Sample of 1000 for testing
product_embeddings = get_bert_embeddings_batch(products_data_copy['combined_text'].tolist(), batch_size=16)

# Store embeddings in DataFrame
products_data_copy['embedding'] = list(product_embeddings)  # Each embedding as a list for easy storage
products_data_copy.head()


100%|██████████| 5186/5186 [4:40:39<00:00,  3.25s/it]


,product_id,product_description,department_key,class_key,subclass_key,sector,brand_key,shelf_level1,shelf_level2,shelf_level3,shelf_level4,bio,sugar_free,aspartame_free,gluten_free,halal,casher,eco_friendly,local_french,artificial_coloring_free,taste_enhancer_free,naturality,antibiotic_free,reduced_sugar,vegetarian,pesticide_free,grain_free,no_added_sugar,salt_reduced,nitrite_free,fed_without_ogm,no_added_salt,no_artificial_flavours,porc,vegan,frozen,fat_free,reduced_fats,fresh,alcool,lactose_free,phenylalanine_free,palm_oil_free,ecoscore,produits_du_monde,regional_product,national_brand,first_price_brand,carrefour_brand,combined_text,embedding
0,Product_33508,PET 1L LORINA TONIC ARTISANAL,Department_10,Class_1000,SubClass_10000,PGC,LORINA,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0,Product_33508 PET 1L LORINA TONIC ARTISANAL Se...,"[-0.43221116, -0.020506142, 0.27752018, 0.0597..."
1,Product_15347,"1,5L LIMO PLANCOET",Department_10,Class_1000,SubClass_10000,PGC,PLANCOET,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,1,0,0,"Product_15347 1,5L LIMO PLANCOET Sector: PGC B...","[-0.35986277, -0.016867476, 0.26125276, 0.1069..."
2,Product_80604,"PET 1,5L LIMONADE SIMPL",Department_10,Class_1000,SubClass_10000,PGC,SIMPL,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,1,0,"Product_80604 PET 1,5L LIMONADE SIMPL Sector: ...","[-0.30704668, 0.016904986, 0.30627796, 0.08935..."
3,Product_81740,PET 1.5L LIMONADE LIGHT OGEU,Department_10,Class_1000,SubClass_10000,PGC,OGEU,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,1,0,0,Product_81740 PET 1.5L LIMONADE LIGHT OGEU Sec...,"[-0.47089338, 0.0709071, 0.19771996, 0.0344978..."
4,Product_82516,1L LIMONADE ARTISANAL LORINA,Department_10,Class_1000,SubClass_10000,PGC,LORINA,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0,Product_82516 1L LIMONADE ARTISANAL LORINA Sec...,"[-0.42436406, -0.029012717, 0.26775962, 0.0426..."


### Load

In [ ]:
products_data_copy.to_csv(f'{trainFolder}/products_embe.csv', index=False)

In [ ]:
products_data_copy.head()

,product_id,embedding
0,Product_33508,[-4.32211161e-01 -2.05061417e-02 2.77520180e-...
1,Product_15347,[-3.59862775e-01 -1.68674756e-02 2.61252761e-...
2,Product_80604,[-3.07046682e-01 1.69049855e-02 3.06277961e-...
3,Product_81740,[-4.70893383e-01 7.09071010e-02 1.97719961e-...
4,Product_82516,[-4.24364060e-01 -2.90127173e-02 2.67759621e-...


In [ ]:
import numpy as np
import pandas as pd


train_data_with_embeddings = train_data.merge(
    products_data_copy[['product_id', 'embedding']],
    on='product_id',
    how='inner'
)


def aggregate_embeddings(embeddings):
    embeddings = embeddings.apply(lambda x: np.fromstring(x[1:-1], sep=' ', dtype=float))
    return np.mean(np.stack(embeddings), axis=0)


user_embeddings_df = train_data_with_embeddings.groupby('customer_id')['embedding'].apply(aggregate_embeddings)
user_embeddings = user_embeddings_df.to_dict()
train_data_with_embeddings.head()

,date,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel,embedding
0,2023-11-15,Transaction_1730125,Household_39,Product_5362,0,Store_2,0,1.0,DRIVE,MOBILE_APP,[-2.99103469e-01 3.67448814e-02 2.09850088e-...
1,2022-07-20,Transaction_1560535,Household_39,Product_67174,0,Store_2,0,2.0,DRIVE,WEBSITE,[-8.82838964e-02 2.86079422e-02 2.54141301e-...
2,2022-07-20,Transaction_1560535,Household_39,Product_82254,0,Store_2,0,2.0,DRIVE,WEBSITE,[-4.20914948e-01 1.13066286e-02 2.97245324e-...
3,2023-11-15,Transaction_1730125,Household_39,Product_3895,0,Store_2,0,1.0,DRIVE,MOBILE_APP,[-1.63046032e-01 3.51533629e-02 3.45103770e-...
4,2022-07-20,Transaction_1560535,Household_39,Product_34014,0,Store_2,0,1.0,DRIVE,WEBSITE,[-2.35846683e-01 -4.98787239e-02 1.25820220e-...


### Create Recommendations based on User Embeddings

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Assuming products_data_copy['embedding'] contains strings like '[-0.123 0.456 ...]'
# Convert them to NumPy arrays
products_data_copy['embedding'] = products_data_copy['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=' ', dtype=float))

all_product_embeddings = np.vstack(products_data_copy['embedding'].values)

def recommend_products_for_user(user_id, user_embeddings, all_product_embeddings, top_k=10):
    """Recommends products for a user based on cosine similarity of embeddings."""
    user_embedding = user_embeddings.get(user_id)
    if user_embedding is None:
        return []

    user_embedding = np.array(user_embedding).reshape(1, -1)

    similarities = cosine_similarity(user_embedding, all_product_embeddings)

    recommended_indices = similarities[0].argsort()[-top_k:][::-1]
    return recommended_indices

# Example usage
user_id = 'Household_9504'
recommended_indices = recommend_products_for_user(user_id, user_embeddings, all_product_embeddings)
recommended_products = products_data.iloc[recommended_indices]
print(recommended_products[['product_id']])

          product_id
72939  Product_50854
18398  Product_50393
27379  Product_71613
65702  Product_75459
72754  Product_81130
71454  Product_39431
39310  Product_76944
67171  Product_23499
67176  Product_30592
18429  Product_29675


### **Evaluation**

In [ ]:
# Group by customer_id to create ground truth
ground_truth = (
    test_data.groupby("customer_id")["product_id"]
    .apply(set)
    .to_dict()
)


In [ ]:
top_k = 10
predicted_recommendations = {}

for customer_id in ground_truth.keys():
    recommended_indices = recommend_products_for_user(
        customer_id, user_embeddings, all_product_embeddings, top_k=top_k
    )
    predicted_products = products_data.iloc[recommended_indices]["product_id"].tolist()
    predicted_recommendations[customer_id] = set(predicted_products)

In [ ]:
def calculate_metrics(predicted, ground_truth, k):
    precision_list = []
    recall_list = []
    hit_rate = 0

    for customer_id, actual_products in ground_truth.items():
        if customer_id not in predicted:
            continue

        recommended_products = predicted[customer_id]

        # Calculate true positives
        true_positives = len(recommended_products & actual_products)

        # Precision@k
        precision = true_positives / k
        precision_list.append(precision)

        # Recall@k
        recall = true_positives / len(actual_products) if len(actual_products) > 0 else 0
        recall_list.append(recall)

        # Hit Rate@k
        if true_positives > 0:
            hit_rate += 1

    # Aggregate metrics
    avg_precision = sum(precision_list) / len(precision_list)
    avg_recall = sum(recall_list) / len(recall_list)
    hit_rate /= len(ground_truth)

    return avg_precision, avg_recall, hit_rate


In [ ]:
avg_precision, avg_recall, hit_rate = calculate_metrics(
    predicted_recommendations, ground_truth, k=top_k
)

print(f"Precision@{top_k}: {avg_precision:.4f}")
print(f"Recall@{top_k}: {avg_recall:.4f}")
print(f"Hit Rate@{top_k}: {hit_rate:.4f}")


Precision@10: 0.0015
Recall@10: 0.0010
Hit Rate@10: 0.0139


## Create baseline frequency approach

### Create frequency dataframe

In [ ]:
# Create a new DataFrame with customer_id, product_id, and count
frequency_df = train_data.groupby(['customer_id', 'product_id'])['transaction_id'].count().reset_index(name='frequency')
frequency_df.sample(4)

,customer_id,product_id,frequency
1172477,Household_413,Product_69751,5
2167016,Household_6821,Product_45871,1
707475,Household_2878,Product_42649,6
2409904,Household_7495,Product_23747,3


In [ ]:
# Rank the frequency for each customer in descending order
frequency_df['rank'] = frequency_df.groupby('customer_id')['frequency'].rank(ascending=False, method="first").astype("Int64")

### Explore results

Let's see the top 10 most purchased products bought by Household_1 in 2022 and 2023.

In [ ]:
frequency_df[frequency_df.customer_id == "Household_1"].sort_values(by = "rank").head(10)

,customer_id,product_id,frequency,rank
347,Household_1,Product_57942,12,1
390,Household_1,Product_64067,11,2
421,Household_1,Product_67459,10,3
504,Household_1,Product_7783,8,4
97,Household_1,Product_21613,7,5
116,Household_1,Product_24334,7,6
541,Household_1,Product_833,7,7
440,Household_1,Product_7006,6,8
6,Household_1,Product_1128,5,9
54,Household_1,Product_170,5,10


And let's see what he actually bought in 2024

In [ ]:
test_data[test_data.customer_id == "Household_1"]

,transaction_id,customer_id,product_id
69457,Transaction_2024_17216,Household_1,Product_45719
69458,Transaction_2024_17216,Household_1,Product_170
69459,Transaction_2024_17216,Household_1,Product_24334
69460,Transaction_2024_17216,Household_1,Product_64067
407363,Transaction_2024_17216,Household_1,Product_35730
407364,Transaction_2024_17216,Household_1,Product_57011
407365,Transaction_2024_17216,Household_1,Product_57942
407366,Transaction_2024_17216,Household_1,Product_73323
813971,Transaction_2024_17216,Household_1,Product_66850
813972,Transaction_2024_17216,Household_1,Product_49682


As you can see here $4$ out of $10$ most frequently purchased products by *Household_1* were re-purchased again by him during his first purchase of 2024

 Create output file (only 10 recommendations per customer)

In [ ]:
# Here we keep only the top 10 recommendation and we drop the frequency as it will not be used in later steps
top_10_recommendations = frequency_df[frequency_df['rank'] <= 10].drop(columns = "frequency")

In [ ]:
# show the top 10 recommendations for customer Household_1 and Household_2

top_10_recommendations[top_10_recommendations.customer_id.isin(["Household_1", "Household_2"])].sort_values(by = ["customer_id", "rank"])

,customer_id,product_id,rank
347,Household_1,Product_57942,1
390,Household_1,Product_64067,2
421,Household_1,Product_67459,3
504,Household_1,Product_7783,4
97,Household_1,Product_21613,5
116,Household_1,Product_24334,6
541,Household_1,Product_833,7
440,Household_1,Product_7006,8
6,Household_1,Product_1128,9
54,Household_1,Product_170,10


### Evaluate frequency model on whole test data using hitrate@10

The aim here is to assess how relevant our 10 product recommendations are. To do this, we calculate the percentage of these recommendations that are actually purchased by the customer.

In order to be precise, we describe here the formula for the computation of the score **hitrate@10**.
Let us introduce two vectors:
+ $(a_i)_{1 \le i \le N}$ with $N \in \mathbb{N}^*$ the products purchased by a customer during an order,
+ $(y_i)_{1 \le i \le 10}$ the $10$ predicted products.

Let us remark that $N$ is not necessarily equal to $10$. Then we have the following formula:


$$HitRate@K(a,y) = \frac{1}{\min(N,10)}\sum_{i=1}^{10} \mathbb{1}_{y_i \in \{a_1,...,a_N\}}$$

Let us provide several remarks:
+ We divide by $\min(N,10)$ in order to have a perfect score $1$ when the customer buys less than $10$ products and all these products are predicted.
+ This definition implicitly assumes that all $y_1,...,y_{10}$ are distinct. If some values are identical, the result can be distorted: for example, if the same product is entered $10$ times and this product is purchased, then the score is $1$. In order to avoid such a case, we ask you to have $10$ different products in your prediction.
+ For the private and public score on kaggle, the rank is not taken into account. Nevertheless, we may use the score   **hitrate@K** for $K<10$ after the competition to decide between possible very close groups. So we ask you to give for each product a rank between $1$ and $10$ (i.e. a permutation of $\{1,...,10\}$).

In [ ]:
# Hitrate@10 evaluation function

def hitrate_at_k(true_data: pd.DataFrame,
                 predicted_data: pd.DataFrame,
                 k: int = 10) -> float:
    """
    This function calculates the hitrate at k for the recommendations.
    It assesses how relevant our 10 product recommendations are.
    In other words, it calculates the proportion of recommended products that are actually purchased by the customer.

    Args:
        true_data: a pandas DataFrame containing the true data
            customer_id: the customer identifier
            product_id: the product identifier that was purchased in the test set
        predicted_data: a pandas DataFrame containing the predicted data
            customer_id: the customer identifier
            product_id: the product identifier that was recommended
            rank: the rank of the recommendation. the rank should be between 1 and 10.
        k: the number of recommendations to consider. k should be between 1 and 10.

    Returns:
        The hitrate at k
    """

    data = pd.merge(left = true_data, right = predicted_data, how = "left", on = ["customer_id", "product_id"])
    df = data[data["rank"] <= k]
    non_null_counts = df.groupby('customer_id')['rank'].apply(lambda x: x.notna().sum()).reset_index(name='non_null_count')
    distinct_products_per_customer = data.groupby('customer_id')['product_id'].nunique().reset_index(name='distinct_product_count')
    df = pd.merge(left = distinct_products_per_customer, right = non_null_counts, how = "left", on = "customer_id")
    df["denominator"] = [min(df.iloc[i].distinct_product_count,k) for i in range(len(df))]
    df = df.fillna(0)
    return (df["non_null_count"]/df["denominator"]).mean()

In [ ]:
# Calculate the hitrate at k for k = 10
frequency_model_hitrate_at_10 = hitrate_at_k(test_data,top_10_recommendations,10)
print(f"Hitrate@10 for the frequency model is {frequency_model_hitrate_at_10:.2f}")

Hitrate@10 for the frequency model is 0.04


# Create submission file

The goal of this part is to provide a function that allows you to encode your prediction in a format that is readable by kaggle when you submit it. In particular, this function checks that you have 10 distinct products per customer and that the ranks are some distinct integers between 1 to 10.

In [ ]:
# Create submission file for

# Keep only the top 10 recommendations for Households between 80001 and 100000
prediction = top_10_recommendations[
    top_10_recommendations.customer_id.isin(
            [
                f"Household_{i}" for i in range( ,100001)
            ]
        )
    ]

# Print the solution
prediction.head()

In [ ]:
def process_and_format_prediction(df):
    # Remplacement des caractères invalides dans les noms de colonnes
    df.columns = df.columns.str.replace('+AF8-', '_', regex=False)
    df = df.replace(r'\+AF8-', '_', regex=True)

    # Nettoyage des colonnes 'customer_id', 'product_id', et 'transaction_id'
    if 'customer_id' in df.columns and df['customer_id'].dtype == 'object':
        df['customer_id'] = df['customer_id'].str.extract('(\d+)').fillna(11).astype(int)
    if 'product_id' in df.columns and df['product_id'].dtype == 'object':
        df['product_id'] = df['product_id'].str.extract('(\d+)').fillna(11).astype(int)
    if 'transaction_id' in df.columns and df['transaction_id'].dtype == 'object':
        df['transaction_id'] = df['transaction_id'].str.replace(r'\D', '', regex=True).fillna(11).astype(int)

    df['id'] = df.index
    df = df[['id'] + [col for col in df.columns if col != 'id']]

    if 'customer_id' not in df.columns or 'product_id' not in df.columns:
        raise ValueError("true_data must contain 'customer_id' and 'product_id' columns")

    # Grouper par customer_id et concaténer les valeurs des produits et des ranks
    prediction_grouped = df.groupby('customer_id').agg({
        'id': 'first',  # Prend la première valeur de 'id'
        'product_id': lambda x: ','.join(map(str, x)),  # Concatène les product_id en chaîne de caractères
        'rank': lambda x: ','.join(map(str, x))  # Concatène les ranks en chaîne de caractères
    }).reset_index()

    # Supprimer la colonne 'id' si elle existe
    if 'id' in prediction_grouped.columns:
        prediction_grouped = prediction_grouped.drop(columns=['id'])

    # Filtrer les données
    prediction_grouped = prediction_grouped[prediction_grouped['customer_id'] != 11]
    prediction_grouped.insert(0, 'id', range(len(prediction_grouped)))

       # Vérification des rangs et des doublons
    for index, row in prediction_grouped.iterrows():
        # Vérifier les ranks
        ranks = list(map(int, row['rank'].split(',')))
        if sorted(ranks) != list(range(1, 11)):  # Vérifie que les rangs sont distincts de 1 à 10
            print("Doublon détecté. Les rangs doivent être distincts (de 1 à 10) pour chacun des 10 produits prédits pour un client.\n")
            return None
        # Vérifier les doublons de produits
        products = row['product_id'].split(',')
        if len(products) != len(set(products)):  # Si des doublons sont présents dans les produits
            print("Doublon détecté. Il doit y avoir 10 produits différents par client.\n")
            return None


    return prediction_grouped
prediction_grouped=process_and_format_prediction(prediction)
print(prediction_grouped)


In [ ]:
# Create a .csv file to submit on kaggle
# A lancer en local sur votre ordinateur
prediction_grouped.to_csv('submission/submission_list_bert.csv', index=False) ####